# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [11]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [19]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
 
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [12]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

stream = openai.chat.completions.create(model=MODEL_GPT, messages=[{"role":"user", "content":question}], stream = True)
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

The code you've provided is a Python expression that uses several concepts and constructs from the language. Let's break it down step by step:

1. **Set Comprehension**: 
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension. This creates a set—a collection of unique elements—by iterating over the `books` collection.
   - `book.get("author")` retrieves the value associated with the key `"author"` from each `book` dictionary. The `get` method is used instead of directly accessing the dictionary with square brackets to avoid a `KeyError` if the `"author"` key does not exist.

2. **Filtering**:
   - The `if book.get("author")` condition filters out any `book` dictionaries that do not have an `"author"` key or have an `author` value that is falsy (which in Python would include `None`, empty strings, etc.). Only `book` dictionaries with valid, non-empty author values will contribute to the resulting set.

3. **Yielding**:
   - The `yield from` statement is used in a generator function to yield all values from the iterable (in this case, the set created by the comprehension). 
   - This means that all unique authors that were gathered from the `books` collection will be yielded one by one when this generator is iterated over.

### Purpose of the Code

The purpose of this code is to create a generator that yields all unique authors from the `books` collection where an author is specified. It ensures that only non-empty author names are included and that there are no duplicates because sets inherently prevent duplicate entries.

### Example

For example, if you have a list of books structured like this:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
]


The output of the code will be:
- "Author A"
- "Author B"

When you iterate over this generator, you would retrieve each unique author specified in the books without duplicates and ignoring any books that do not specify an author.

In [20]:
# Get Llama 3.2 to answer
payload = {
        "model": MODEL_LLAMA,
        "messages": question,
        "stream": False
    }

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

KeyError: 'message'